# 🌍 NLLB-200 Translation Service — Lightning AI

Lance le service de traduction NLLB-200 sur GPU Lightning AI et se connecte automatiquement à votre application.

**Prérequis :**
1. Créer un compte sur [lightning.ai](https://lightning.ai) (15 crédits GPU/mois offerts)
2. Créer un nouveau **Studio** → choisir un GPU (A10G recommandé)
3. Ouvrir ce notebook dans le Studio

**Une seule étape :** `Run All`

> **Avantage Lightning AI** : Le stockage est persistant — le modèle NLLB se télécharge une seule fois (~/50 Go disponibles)

In [ ]:
# ── Cellule 1 : Configuration ─────────────────────────────────────────────
# URL de votre application (modifier avec l'IP de votre VM)
VM_URL = "http://144.126.203.233:3001"
# Token de sécurité (doit correspondre à COLAB_SECRET dans le backend)
COLAB_SECRET = "docuLens2024"
# Port du service
PORT = 5001
# Chemin de stockage persistant (Lightning AI conserve /teamspace entre sessions)
WORK_DIR = "/teamspace/studios/this_studio"

In [ ]:
# ── Cellule 2 : Installation ──────────────────────────────────────────────
import subprocess, sys, os

print('📦 Installation des dépendances...')
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
    'transformers', 'sentencepiece', 'fastapi', 'uvicorn[standard]',
    'nest_asyncio', 'accelerate'], check=True)

# Cloudflare tunnel (pour exposer le service publiquement)
if not os.path.exists('/usr/local/bin/cloudflared'):
    subprocess.run([
        'curl', '-sL',
        'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64',
        '-o', '/usr/local/bin/cloudflared'
    ], check=True)
    subprocess.run(['chmod', '+x', '/usr/local/bin/cloudflared'], check=True)
    print('✅ cloudflared installé')
else:
    print('✅ cloudflared déjà présent')

import torch
gpu = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU (pas de GPU détecté)'
print(f'✅ Installation terminée — GPU : {gpu}')

In [ ]:
# ── Cellule 3 : Service FastAPI ───────────────────────────────────────────
import os
os.makedirs(WORK_DIR, exist_ok=True)

service_code = '''
import os, logging, asyncio
from contextlib import asynccontextmanager
from typing import List
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger(__name__)

# Sur Lightning AI, le cache HuggingFace est persistant entre sessions
MODEL_NAME = os.getenv("NLLB_MODEL", "facebook/nllb-200-distilled-600M")
MAX_LENGTH = int(os.getenv("NLLB_MAX_LENGTH", "512"))
BATCH_SIZE = int(os.getenv("NLLB_BATCH_SIZE", "32"))
PORT       = int(os.getenv("TRANSLATION_PORT", "5001"))

state = {"model": None, "tokenizer": None, "device": "cpu", "ready": False}

def load_model():
    from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
    import torch
    device = "cuda" if torch.cuda.is_available() else "cpu"
    log.info(f"Chargement {MODEL_NAME} sur {device.upper()}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )
    model.to(device).eval()
    state.update({"tokenizer": tokenizer, "model": model, "device": device, "ready": True})
    log.info(f"Modele pret sur {device.upper()}")

@asynccontextmanager
async def lifespan(app):
    loop = asyncio.get_event_loop()
    await loop.run_in_executor(None, load_model)
    yield

app = FastAPI(title="NLLB Translation Service", lifespan=lifespan)
app.add_middleware(CORSMiddleware, allow_origins=["*"], allow_methods=["*"], allow_headers=["*"])

class TranslateRequest(BaseModel):
    texts: List[str]
    src_lang: str = "fra_Latn"
    tgt_lang: str = "eng_Latn"

def translate_batch(texts, src_lang, tgt_lang):
    import torch
    tokenizer, model, device = state["tokenizer"], state["model"], state["device"]
    results = []
    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i+BATCH_SIZE]
        non_empty_idx  = [j for j, t in enumerate(batch) if t.strip()]
        non_empty_text = [batch[j] for j in non_empty_idx]
        batch_results  = [""] * len(batch)
        if non_empty_text:
            tokenizer.src_lang = src_lang
            inputs = tokenizer(
                non_empty_text, return_tensors="pt",
                padding=True, truncation=True, max_length=MAX_LENGTH
            ).to(device)
            forced_bos = tokenizer.convert_tokens_to_ids(tgt_lang)
            with torch.no_grad():
                output_ids = model.generate(
                    **inputs, forced_bos_token_id=forced_bos,
                    max_length=MAX_LENGTH, num_beams=4
                )
            translated = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
            for idx, t in zip(non_empty_idx, translated):
                batch_results[idx] = t
        results.extend(batch_results)
    return results

@app.get("/health")
def health():
    return {"status": "ready" if state["ready"] else "loading", "model": MODEL_NAME, "device": state["device"]}

@app.post("/translate")
async def translate(req: TranslateRequest):
    if not state["ready"]:
        raise HTTPException(503, detail="Modele en chargement")
    if req.src_lang == req.tgt_lang:
        raise HTTPException(400, detail="src_lang et tgt_lang doivent etre differents")
    loop = asyncio.get_event_loop()
    translations = await loop.run_in_executor(None, translate_batch, req.texts, req.src_lang, req.tgt_lang)
    return {"translations": translations, "src_lang": req.src_lang, "tgt_lang": req.tgt_lang, "count": len([t for t in translations if t])}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="info")
'''

service_path = os.path.join(WORK_DIR, 'translation_service.py')
with open(service_path, 'w') as f:
    f.write(service_code)
print(f'✅ Service écrit dans {service_path}')

In [ ]:
# ── Cellule 4 : Lancement uvicorn ─────────────────────────────────────────
import subprocess, time, requests, threading, os

env = os.environ.copy()
env['TRANSLATION_PORT'] = str(PORT)

proc = subprocess.Popen(
    ['python', '-m', 'uvicorn', 'translation_service:app',
     '--host', '0.0.0.0', '--port', str(PORT)],
    cwd=WORK_DIR,
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, env=env
)
threading.Thread(
    target=lambda: [print(l.decode('utf-8','replace'), end='') for l in proc.stdout],
    daemon=True
).start()

print('⏳ Chargement du modèle NLLB-200...')
print('   Lightning AI : modèle mis en cache → démarrage rapide après la 1ère fois')

for i in range(120):
    time.sleep(3)
    try:
        r = requests.get(f'http://localhost:{PORT}/health', timeout=2)
        if r.json().get('status') == 'ready':
            d = r.json()
            print(f'\n✅ Modèle prêt sur {d["device"].upper()} — {d["model"]}')
            break
    except:
        pass
else:
    print('❌ Timeout — vérifiez les logs ci-dessus')

In [ ]:
# ── Cellule 5 : Tunnel Cloudflare + connexion automatique à la VM ─────────
import subprocess, re, requests, time

tunnel = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', f'http://localhost:{PORT}'],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT
)

PUBLIC_URL = None
print('⏳ Ouverture du tunnel Cloudflare...')
for line in tunnel.stdout:
    text = line.decode('utf-8', errors='replace')
    match = re.search(r'https://[\w-]+\.trycloudflare\.com', text)
    if match:
        PUBLIC_URL = match.group(0)
        break

print(f'\n🌐 URL publique : {PUBLIC_URL}')

# Connexion automatique à la VM
print('\n📡 Connexion à l\'application...')
connected = False
for attempt in range(5):
    try:
        r = requests.post(
            f'{VM_URL}/api/config/translation-url',
            json={'url': PUBLIC_URL, 'secret': COLAB_SECRET},
            timeout=10
        )
        if r.status_code == 200:
            print('\n' + '=' * 60)
            print('🚀 GPU Lightning AI connecté à votre application !')
            print('   Le badge GPU passe au vert dans l\'app')
            print('=' * 60)
            connected = True
            break
    except Exception as e:
        print(f'   Tentative {attempt+1}/5... ({e})')
        time.sleep(3)

if not connected:
    print('\n⚠️  Connexion automatique échouée.')
    print(f'   → Copiez cette URL manuellement dans l\'app : {PUBLIC_URL}')

In [ ]:
# ── Cellule 6 : Garder la session active ──────────────────────────────────
# Lightning AI coupe les sessions sans activité après ~4h
# Cette cellule maintient le service en vie
import time

print('✅ Service actif — Lightning AI GPU en cours d\'utilisation')
print(f'   URL : {PUBLIC_URL}')
print('   Arrêt propre : Kernel → Interrupt')
print()

start = time.time()
while True:
    elapsed = int(time.time() - start)
    h, m = divmod(elapsed // 60, 60)
    try:
        r = requests.get(f'http://localhost:{PORT}/health', timeout=2)
        status = r.json().get('status', '?')
        device = r.json().get('device', '?')
    except:
        status = 'unreachable'
        device = '?'
    print(f'   ⏱ {h:02d}h{m:02d}m | {device.upper()} | {status}', end='\r')
    time.sleep(30)